# GEFS Combiner Script
## Robert Fritzen - NIU WCS

This script (notebook) is used to generate WRF intermediate files from the GEFS Reforecast Set.

## Imports
* xarray
* xesmf
* numpy
* pygrib
* pywinter (pip3 install pywinter -> Needs gfortran)

In [1]:
import xarray as xr
import xesmf as xe
import numpy as np
from datetime import datetime, timedelta
import os
import sys
import pygrib
import pickle

# pip3 install pywinter (Need gfortran installed)
from pywinter import winter

<frozen importlib._bootstrap>:219: UserWarning: Warning: ecCodes 2.19.1 or higher is recommended. You are running 2.17.0.


## Settings

In [7]:
# Format: YYYYMMDDHH
start_date = "2019051900"
end_date = "2019052300"
#end_date = "2019052300"

# Which hour of the GEFS reforecast set to use as initialization, intervals of 24h only
fhr = 24

# Time delta of GEFS (6 hours)
dt = 6 #NOTE: 3 hr deltas are available from days 1 - 10 only

# Which member of the GEFS reforecast set to use (Options are c00, p01, p02, p03, and p04)
#  There are other ensemble perturbation members p05 - p10 available on the Wednesday runs only.
dataMember = "c00"

# What grid scale we should use (Note: You MUST pick the largest grid from all variables for the regridder)
#  This variable is in degrees
gridSize = 0.5

# Data source (Leave this as is unless the source is moved (Will require an edit of the next block))
dataSrc = "https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/"

# Variable List
These are the variables to be processed, the arguments are as follows:

* sLevels: Special list to store soil variable levels contained in the set. Most commonly are the standard WRF levels (0-10cm, 10-40cm, 40-100cm, 100-200cm).
* pres: A list of the vertical pressure levels for this GRIB set (Units of Pa).
* gridDefs: A dictionary object with constant objects (non-time) that contain the grid definitions, usually sets provide this through the land surface file. If not, find a "small" file with the grid and use that.

Within the Variable dictionary:
* path: The path to the files, in a list format in case multiple files are used (GEFS)
* grid: The grid spacing of the GRIB data
* iFileName: The variable name in the intermediate file (Check the WRF documentation for the table)
* fillZNan: If there are missing vertical levels, should these be filled with NANs?
* is3D: Is the variable a 3D field?
* isSoilVar: Is this a soil variable? (Note: Do not set is3D to True for soil)
* isLandSeaMask: A special flag to indicate that this is the land-sea mask variable.
* desiredLevel: If we are only looking to pull a specific vertical level out, define it here.
* NANOverWater: Set the value of the variable to np.nan over non-land areas (Water, uses the land-sea mask)
* NeedFlipUD: Set to True if the incoming data are flipped on rendering axis

There are a few extra special flags that can be set:
* isSoilMoisture: Set to True for the one variable that contains the soil moisture (Must be isSoilVar also True)
* isSoilTemperature: Set to True for the one variable that contains the soil temperature (Must be isSoilVar also True)

In [3]:
sLevels = ['000010', '010040', '040100', '100200']

pres = [100000., 97500., 95000., 92500., 90000., 85000., 80000., 70000., 60000., 50000., 40000., 30000., 25000., 
        20000., 15000., 10000., 7000., 5000., 3000., 2000., 1000., 500., 300., 200., 100.]

gridDefs = {
    "0.25": 
        {'url': "https://noaa-gefs-retrospective.s3.amazonaws.com/landsfc.pgrb2.0p25"},
    "0.5": 
        {'url': "https://noaa-gefs-retrospective.s3.amazonaws.com/landsfc.pgrb2.0p50"},
}

regridders = {}

variables = {
    "PMSL":
        {'path': ["pres_msl_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'PMSL',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},     
    
    "TT":
        {'path': ["tmp_pres_[timestring]_" + dataMember + ".grib2", 
                  "tmp_pres_abv700mb_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25, 0.5],
         'iFileName' : 'TT',
         'fillZNan': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},

    "UU":
        {'path': ["ugrd_pres_[timestring]_" + dataMember + ".grib2", 
                  "ugrd_pres_abv700mb_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25, 0.5],
         'iFileName' : 'UU',
         'fillZNan': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},  

    "VV":
        {'path': ["vgrd_pres_[timestring]_" + dataMember + ".grib2", 
                  "vgrd_pres_abv700mb_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25, 0.5],
         'iFileName' : 'VV',
         'fillZNan': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},

    "SPECHUMD":
        {'path': ["spfh_pres_[timestring]_" + dataMember + ".grib2", 
                  "spfh_pres_abv700mb_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25, 0.5],
         'iFileName' : 'SPECHUMD',
         'fillZNan': True,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},   

    "GHT":
        {'path': ["hgt_pres_[timestring]_" + dataMember + ".grib2", 
                  "hgt_pres_abv700mb_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25, 0.5],
         'iFileName' : 'GHT',
         'fillZNan': False,
         'is3D': True,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},     
    
    "T2":
        {'path': ["tmp_2m_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'TT',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},    
    
    "Q2":
        {'path': ["spfh_2m_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'SPECHUMD',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},   
    
    "U10":
        {'path': ["ugrd_hgt_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'UU',
         'desiredLevel': 10,
         'fillZNan': False,
         'is3D': False,
         'isLandSeaMask': False,
         'isSoilVar': False,
         'NANOverWater': False,
         'NeedFlipUD': True},     

    "V10":
        {'path': ["vgrd_hgt_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'VV',
         'desiredLevel': 10,
         'fillZNan': False,
         'is3D': False,
         'isLandSeaMask': False,
         'isSoilVar': False,
         'NANOverWater': False,
         'NeedFlipUD': True},    

    "PSFC":
        {'path': ["pres_sfc_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'PSFC',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},              

    "SM":
        {'path': ["soilw_bgrnd_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'SM',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': True,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'isSoilMoisture': True,
         'isSoilTemperature': False,
         'NANOverWater': True,
         'NeedFlipUD': True}, 
    
    "ST":
        {'path': ["tsoil_bgrnd_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'ST',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': True,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'isSoilMoisture': False,
         'isSoilTemperature': True,
         'NANOverWater': True,
         'NeedFlipUD': True},     
    
    # LANDSEA is a special case, handled within the loop
    "LANDSEA":
        {'path': None,
         'grid': None,
         'iFileName': 'LANDSEA',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': True,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},
    
    # To-Do Note: This comes in gpm, not m, check if ok?
    "SOILHGT":
        {'path': ["hgt_sfc_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'SOILHGT',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},     
    
    "SKINTEMP":
        {'path': ["tmp_sfc_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'SKINTEMP',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': False,
         'NeedFlipUD': True},                    
    
    "SNOW":
        {'path': ["weasd_sfc_[timestring]_" + dataMember + ".grib2"],
         'grid': [0.25],
         'iFileName' : 'SNOW',
         'fillZNan': False,
         'is3D': False,
         'isSoilVar': False,
         'isLandSeaMask': False,
         'desiredLevel': None,
         'NANOverWater': True,
         'NeedFlipUD': True},                  
}

# Function definitions
def grib_to_xs(grib, vName):
    vals = grib.values
    print(" - Converting GRIB to XArray of type " + str(type(vals)))
    if(np.ma.isMaskedArray(vals)):
        print(" -- Converting Masked Numpy Array to Standard Numpy Array")
        # Convert the masked array to a standard np array
        vals = np.ma.filled(vals, np.nan)
    
    arr = xr.DataArray(vals)
    arr = arr.astype('float32')
    arr = arr.rename({'dim_0':'lat', 'dim_1':'lon'})
    xs = arr.to_dataset(name=vName)
    
    return xs

def get_bounds(arr, gridSize):    
    lonMin = np.nanmin(arr["distinctLongitudes"])
    latMin = np.nanmin(arr["distinctLatitudes"])
    lonMax = np.nanmax(arr["distinctLongitudes"])
    latMax = np.nanmax(arr["distinctLatitudes"])
    
    sizeLon = len(arr["distinctLongitudes"])
    sizeLat = len(arr["distinctLatitudes"])
    
    bounds = {}
    
    bounds["lon"] = arr["distinctLongitudes"]
    bounds["lat"] = arr["distinctLatitudes"]
    bounds["lon_b"] = np.linspace(lonMin-(gridSize/2), lonMax+(gridSize/2), sizeLon+1)
    bounds["lat_b"] = np.linspace(latMin-(gridSize/2), latMax+(gridSize/2), sizeLat+1).clip(-90, 90)
    
    return bounds

## Pre-Run Variables
Run this code block once you have run the above settings block. This will assign additional variables based on your choices above.

In [8]:
print("Running Setting Tests")
# How many files per day
hourCt = 24 / dt

# Convert the start/end times into python datetime objects
print(" - Testing incoming time settings")
start_tm = datetime.strptime(start_date, '%Y%m%d%H')
end_tm = datetime.strptime(end_date, '%Y%m%d%H')

if(fhr % 24 != 0):
    raise NameError("Error: fhr must be an interval of 24.")

target_date_file = start_tm - timedelta(hours=fhr)

if(int(dataMember[-2:]) > 4 and target_date_file.weekday() != 2):
    raise NameError("Error: p05 - p10 are only available when (start_date - fhr) is a Wednesday")

# The list of times we need to grab
timeList = []

print(" - Setting time list arrays")
cur_tm = start_tm
while cur_tm <= end_tm:
    hDelta = int((cur_tm - target_date_file).total_seconds() / 3600)
    
    timeList.append(hDelta)
    cur_tm += timedelta(hours=dt)
    
print(timeList)

# Determine which day block we need from fhr
print(" - Setting day block arrays")
dayBlocks = []
if(min(timeList) < 240):
    dayBlocks.append("Days%3A1-10")
if(max(timeList) >= 240):
    dayBlocks.append("Days%3A10-16" if start_date_file.weekday() != 2 else "Days%310-35")

print(dayBlocks)

print(" - Collecting soil Z parameter")
soilZ = len(sLevels)

# Collect the grid files
print(" - Collecting grids, defining bounds...")
for key, item in gridDefs.items():
    lnk = item['url']
    fname = lnk[lnk.rfind("/")+1:]
    if not os.path.exists(fname):
        os.system("wget " + lnk + " " + fname)
    gridGrib = pygrib.open(fname)
    gLats = gridGrib[1]["distinctLatitudes"]
    gLons = gridGrib[1]["distinctLongitudes"]
    print(key)
    item["bounds"] = get_bounds(gridGrib[1], float(key))

print(" - Defining regridders")
# Loop through the variable list, gather all "regridders" needed
for key, item in variables.items():
    if item["grid"] is not None:
        for g in item["grid"]:
            if(g != gridSize):
                if not str(g) in regridders:
                    print("Defining " + str(g))
                    regridders[str(g)] = xe.Regridder(gridDefs[str(g)]['bounds'], 
                                                      gridDefs[str(gridSize)]['bounds'], 
                                                      'conservative')
                else:
                    print("Key " + str(g) + " is already defined, skipping.")
                
print("Done.")

Running Setting Tests
 - Testing incoming time settings
 - Setting time list arrays
[24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120]
 - Setting day block arrays
['Days%3A1-10']
 - Collecting soil Z parameter
 - Collecting grids, defining bounds...
0.25
0.5
 - Defining regridders
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Key 0.25 is already defined, skipping.
Done.


In [5]:
print(gridDefs)
lat = gridDefs[str(gridSize)]['bounds']['lat']
lon = gridDefs[str(gridSize)]['bounds']['lon']
print(lat)
print(lon)
print("\n\n")
print(regridders)

{'0.25': {'url': 'https://noaa-gefs-retrospective.s3.amazonaws.com/landsfc.pgrb2.0p25', 'bounds': {'lon': array([0.0000e+00, 2.5000e-01, 5.0000e-01, ..., 3.5925e+02, 3.5950e+02,
       3.5975e+02]), 'lat': array([ 90.  ,  89.75,  89.5 ,  89.25,  89.  ,  88.75,  88.5 ,  88.25,
        88.  ,  87.75,  87.5 ,  87.25,  87.  ,  86.75,  86.5 ,  86.25,
        86.  ,  85.75,  85.5 ,  85.25,  85.  ,  84.75,  84.5 ,  84.25,
        84.  ,  83.75,  83.5 ,  83.25,  83.  ,  82.75,  82.5 ,  82.25,
        82.  ,  81.75,  81.5 ,  81.25,  81.  ,  80.75,  80.5 ,  80.25,
        80.  ,  79.75,  79.5 ,  79.25,  79.  ,  78.75,  78.5 ,  78.25,
        78.  ,  77.75,  77.5 ,  77.25,  77.  ,  76.75,  76.5 ,  76.25,
        76.  ,  75.75,  75.5 ,  75.25,  75.  ,  74.75,  74.5 ,  74.25,
        74.  ,  73.75,  73.5 ,  73.25,  73.  ,  72.75,  72.5 ,  72.25,
        72.  ,  71.75,  71.5 ,  71.25,  71.  ,  70.75,  70.5 ,  70.25,
        70.  ,  69.75,  69.5 ,  69.25,  69.  ,  68.75,  68.5 ,  68.25,
        68.  

In [9]:
# Grab the land-sea mask (Constant)
print("Start")

print("Checking for required directory structure.")
if not os.path.exists(os.getcwd() + "/Input/"):
    print(" - Input directory not found, creating...")
    os.mkdir(os.getcwd() + "/Input/")
if not os.path.exists(os.getcwd() + "/Output/"):
    print(" - Output directory not found, creating...")
    os.mkdir(os.getcwd() + "/Output/")
print("Directory check passed")

print("\n\nGrabbing land - sea mask")
lndsfc = "https://noaa-gefs-retrospective.s3.amazonaws.com/landsfc.pgrb2.0p50"
if not os.path.exists(os.getcwd() + "/Input/landsfc.pgrb2.0p50"):
    os.system("wget " + lndsfc + " -O " + os.getcwd() + "/Input/landsfc.pgrb2.0p50")
lsGrib = pygrib.open(os.getcwd() + "/Input/landsfc.pgrb2.0p50")
landsea = lsGrib[1].values
  
lat = gridDefs[str(gridSize)]['bounds']['lat']
lon = gridDefs[str(gridSize)]['bounds']['lon']

soilM = np.zeros((soilZ, len(lat), len(lon)))
soilT = np.zeros((soilZ, len(lat), len(lon)))

dlat = np.abs(lat[1]-lat[0])
dlon = np.abs(lon[1]-lon[0])

prevStep = None
cleanFiles = False

for t in timeList:
    timeString = (target_date_file + timedelta(hours=t)).strftime("%Y%m%d%H")
    if t < 240:
        dayBlock = dayBlocks[0]
    else:
        # In the event we only have the t > 240 case, don't skip over a 0-length array
        dayBlocks = dayBlock[-1]
        if(prevStep < 240):
            cleanFiles = True
            
    if cleanFiles:
        # Flag is triggered when switching from Day blocks
        for key, item in variables.items():
            print(" - Cleaning " + str(key))
            for idx, iPath in enumerate(item['path']):
                path = os.getcwd() + "/Input/" + iPath
                print("Deleting " + path)
                os.system("rm " + path)
        cleanFiles = False
        
    dataDirectory = dataSrc + target_date_file.strftime("%Y") + "/" + target_date_file.strftime("%Y%m%d%H") + "/" + dataMember + "/" + dayBlock + "/"
    print("Processing " + str(t) + " hr (" + timeString + ")")
    
    print("Checking for existing file...")
    if os.path.exists(os.getcwd() + "/Output/saveif_" + target_date_file.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle"):
        print("Processed data found, loading")
        with open(os.getcwd() + "/Output/saveif_" + target_date_file.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle", 'rb') as handle:
            saveIF = pickle.load(handle)
    else:
        print("No file found, begin processing")
        saveIF = {}

        for key, item in variables.items():
            print(" - Working " + str(key))

            save = []
            if(item['isLandSeaMask'] == True):
                lsv = landsea
                if(item['NeedFlipUD'] == True):
                    lsv = np.flipud(lsv)
                
                saveIF[key] = {'iFileName': item['iFileName'], 
                               'isSoilVariable': item['isSoilVar'], 
                               'is3D': item['is3D'],
                               'values': lsv}
            else:
                for idx, iPath in enumerate(item['path']):
                    iPath = iPath.replace("[timestring]", target_date_file.strftime("%Y%m%d%H"))

                    lnk = dataDirectory + iPath
                    print(iPath + " => " + lnk)
                    if not os.path.exists(os.getcwd() + "/Input/" + iPath):
                        os.system("wget " + lnk + " -O " + os.getcwd() + "/Input/" + iPath)

                    grib = pygrib.open(os.getcwd() + "/Input/" + iPath)
                    try:
                        if(item['desiredLevel'] is not None):
                            subsel = grib.select(forecastTime=t, level=item['desiredLevel'])
                        else:   
                            subsel = grib.select(forecastTime=t)
                    except ValueError:
                        print("An error occured while trying to subset " + iPath)
                        print("Debug Info:\n\n")
                        for i in grib:
                            print(i)
                        sys.exit("Error")
                        
                    if(item['isSoilVar'] == True):
                        # Soil is special, it only comes in the defined levels so just grab it all.
                        for i, g in enumerate(subsel):
                            ds = grib_to_xs(g, key)
                            if(item['grid'][idx] != gridSize):
                                ds = regridders[str(item['grid'][idx])](ds)

                            if(item['NANOverWater'] == True):
                                ds[key].values[np.where(landsea == 0)] = np.nan
                                
                            if(item['NeedFlipUD'] == True):
                                ds[key].values = np.flipud(ds[key].values)                                
                                
                            if(item['isSoilMoisture'] == True):
                                soilM[i] = ds[key].values[i]                                  
                            elif(item['isSoilTemperature'] == True):
                                soilT[i] = ds[key].values[i]  
                            else:
                                print("WARN: Variable defined as neither soil moisture or temperature, skipping")
                            #save.append(ds[key].values) # = np.stack(i.values, axis=0)
                        if(item['isSoilMoisture'] == True):
                            save = soilM
                        elif(item['isSoilTemperature'] == True):
                            save = soilT
                        else:
                            print("WARN: Variable defined as neither soil moisture or temperature, skipping")
                    elif(item['is3D'] == True):
                        # 3D variables are just "stack and save"
                        for i in subsel:   
                            ds = grib_to_xs(i, key)
                            if(item['grid'][idx] != gridSize):
                                ds = regridders[str(item['grid'][idx])](ds)
                              
                            if(item['NANOverWater'] == True):
                                ds[key].values[np.where(landsea == 0)] = np.nan  
                                
                            if(item['NeedFlipUD'] == True):
                                ds[key].values = np.flipud(ds[key].values)                                  
                                
                            save.append(ds[key].values)  # = np.stack(i.values, axis=0)
                    else:
                        # 2D variables are easy since they don't have to worry about a vertical coordinate
                            ds = grib_to_xs(subsel[0], key)
                            if(item['grid'][idx] != gridSize):
                                ds = regridders[str(item['grid'][idx])](ds)    
                                
                            if(item['NANOverWater'] == True):
                                ds[key].values[np.where(landsea == 0)] = np.nan 
                                
                            if(item['NeedFlipUD'] == True):
                                ds[key].values = np.flipud(ds[key].values)                                  
                                
                            save = ds[key].values

                    grib.close()

                saveOut = np.stack(save)
                print(saveOut.shape)
                
                if(item['fillZNan'] == True):
                    print(" -> Fill Empty with NaN Requested...")
                    print("  -> Length of Pressure: " + str(len(pres)))
                    print("  -> Size of Z: " + str(saveOut.shape[0]))

                    fillZ = len(pres) - saveOut.shape[0]
                    print("  -> Need " + str(fillZ) + " NaN Arrays")

                    nans = np.zeros((fillZ, len(lat), len(lon)))  
                    saveOut = np.append(saveOut, nans, 0)

                    print("  -> Size is now: " + str(saveOut.shape[0]))

                saveIF[key] = {'iFileName': item['iFileName'], 
                               'isSoilVariable': item['isSoilVar'], 
                               'is3D': item['is3D'],
                               'values': saveOut}

        with open(os.getcwd() + "/Output/saveif_" + target_date_file.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle", 'wb') as handle:
            pickle.dump(saveIF, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print(" - Data saved to file")
        
    print(" - Done Collecting GRIB data")
    print("\n")
    ## Convert to numpy arrays
    print(" - Converting to INTER: File")
    print("  => Debug Info: ")
    print("  => Pres: " + str(np.array(pres)))
    print("  => sLevels: " + str(np.array(sLevels)))
    
    # Latitude is flipped in GEFS, flip it back before writting the intermediate file.
    winter_geo = winter.Geo0(lat[-1], lon[0], dlat, dlon)
    winter_vars = []
    #
    for key, item in saveIF.items():
        print("  -> Processing " + key + " | Write as: " + item['iFileName'])
        winter_save = None
        if(item['is3D'] == True):
            print("   => 3D Variable")
            winter_save = winter.V3dp(item['iFileName'], item['values'], np.array(pres))
        elif(item['isSoilVariable'] == True):
            print("   => Soil Variable")
            winter_save = winter.Vsl(item['iFileName'], item['values'], np.array(sLevels))
        else:
            print("   => 2D Variable")
            winter_save = winter.V2d(item['iFileName'], item['values'])
        
        winter_vars.append(winter_save)  

    # This is the timestep we are on
    outTime = (target_date_file + timedelta(hours=t)).strftime("%Y-%m-%d_%H")

    # Write the file.
    winter.cinter('FILE', outTime, winter_geo, winter_vars, os.getcwd() + '/Output')  
    
    prevStep = t
    
    print("Done Timestep")
    
print("Cleaning")
lsGrib.close()

for key, item in variables.items():
    print(" - Cleaning " + str(key))
    if(item['path'] is not None):
        for idx, iPath in enumerate(item['path']):
            iPath = iPath.replace("[timestring]", target_date_file.strftime("%Y%m%d%H"))
            path = os.getcwd() + "/Input/" + iPath
            print("Deleting " + path)
            os.system("rm " + path)
    
print("Execution Complete")

Start


Grabbing land - sea mask
Processing 24 hr (2019051900)
Checking for existing file...
No file found, begin processing
 - Working PMSL
pres_msl_2019051800_c00.grib2 => https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/2019/2019051800/c00/Days%3A1-10/pres_msl_2019051800_c00.grib2
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable PMSL as the horizontal dimensions for this dataset.
(361, 720)
 - Working TT
tmp_pres_2019051800_c00.grib2 => https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/2019/2019051800/c00/Days%3A1-10/tmp_pres_2019051800_c00.grib2
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB t

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SPECHUMD as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SPECHUMD as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SPECHUMD as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SPECHUMD as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SPECHUMD as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SPECHUMD as the horizontal dimensions for this d

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SOILHGT as the horizontal dimensions for this dataset.
(361, 720)
 - Working SKINTEMP
tmp_sfc_2019051800_c00.grib2 => https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/2019/2019051800/c00/Days%3A1-10/tmp_sfc_2019051800_c00.grib2
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable SKINTEMP as the horizontal dimensions for this dataset.
(361, 720)
 - Working SNOW
weasd_sfc_2019051800_c00.grib2 => https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/2019/2019051800/c00/Days%3A1-10/weasd_sfc_2019051800_c00.grib2
 - Converting GRIB to XArray of type <class 'numpy.ma.core.MaskedArray'>
 -- Converting Masked Numpy Array to Standard Numpy Array
using dimensions ('lat', 'lon') from data variable SNOW as the horizontal dimensions for this dataset.
(361, 720)
 - Data saved to file
 - Done Collec

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

 - Converting GRIB to XArray of type <class 'numpy.ma.core.MaskedArray'>
 -- Converting Masked Numpy Array to Standard Numpy Array
using dimensions ('lat', 'lon') from data variable ST as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ma.core.MaskedArray'>
 -- Converting Masked Numpy Array to Standard Numpy Array
using dimensions ('lat', 'lon') from data variable ST as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ma.core.MaskedArray'>
 -- Converting Masked Numpy Array to Standard Numpy Array
using dimensions ('lat', 'lon') from data variable ST as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ma.core.MaskedArray'>
 -- Converting Masked Numpy Array to Standard Numpy Array
using dimensions ('lat', 'lon') from data variable ST as the horizontal dimensions for this dataset.
(4, 361, 720)
 - Working LANDSEA
 - Working SOILHGT
hgt_sfc_2019051800_c

...

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable UU as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable UU as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable UU as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable UU as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable UU as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable UU as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XA

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable TT as the horizontal dimensions for this dataset.
 - Converting GRIB to XArray

 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
(16, 361, 720)
 -> Fill Empty with NaN Requested...
  -> Length of Pressure: 25
  -> Size of Z: 16
  -> Need 9 NaN Arrays
  -> Size is now: 25
 - Working GHT
hgt_pres_2019051800_c00.grib2 => https://noaa-gefs-retrospective.s3.amazonaws.com/GEFSv12/reforecast/2019/2019051800/c00/Days%3A1-10/hgt_pres_2019051800_c00.grib2
 - Converting GRIB to XArray of type <class 'numpy.ndarray'>
using dimensions ('lat', 'lon') from data variable GHT as the horizon

 - Data saved to file
 - Done Collecting GRIB data


 - Converting to INTER: File
  => Debug Info: 
  => Pres: [100000.  97500.  95000.  92500.  90000.  85000.  80000.  70000.  60000.
  50000.  40000.  30000.  25000.  20000.  15000.  10000.   7000.   5000.
   3000.   2000.   1000.    500.    300.    200.    100.]
  => sLevels: ['000010' '010040' '040100' '100200']
  -> Processing PMSL | Write as: PMSL
   => 2D Variable
  -> Processing TT | Write as: TT
   => 3D Variable
  -> Processing UU | Write as: UU
   => 3D Variable
  -> Processing VV | Write as: VV
   => 3D Variable
  -> Processing SPECHUMD | Write as: SPECHUMD
   => 3D Variable
  -> Processing GHT | Write as: GHT
   => 3D Variable
  -> Processing T2 | Write as: TT
   => 2D Variable
  -> Processing Q2 | Write as: SPECHUMD
   => 2D Variable
  -> Processing U10 | Write as: UU
   => 2D Variable
  -> Processing V10 | Write as: VV
   => 2D Variable
  -> Processing PSFC | Write as: PSFC
   => 2D Variable
  -> Processing SM | Write as: 

## Debug
If anything goes wrong with the creation of the intermediate files, or there is an input error, you may run the below block to check the saveif files that were written (numpy files containing what's saved in the intermediate file) to inspect the output arrays for errors.

You should also verify your WPS installation using rd_intermediate and then int2nc on the intermediate files that were written by this python program, if either of those two fail, you may have an issue with your installation, or your pyWinter package did not correct

In [60]:
for t in timeList:
    timeString = (target_date_file + timedelta(hours=t)).strftime("%Y%m%d%H")    
    print("Checking for saveif (" + str(t) + ") hr (" + timeString + ")")
    if os.path.exists(os.getcwd() + "/Output/saveif_" + target_date_file.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle"):
        print("Processed data found, loading")
        with open(os.getcwd() + "/Output/saveif_" + target_date_file.strftime("%Y%m%d%H") + "_" + str(timeString) + ".pickle", 'rb') as handle:
            saveIF = pickle.load(handle)
            
            print(saveIF)
            
    print("\n\n\n\n\n\n\n")

Checking for saveif (96) hr (2019051900)
Processed data found, loading
{'PMSL': {'iFileName': 'PMSL', 'isSoilVariable': False, 'is3D': False, 'values': array([[103109.18748394, 103109.14939789, 103108.92527808, ...,
        103109.60056816, 103109.4496898 , 103109.22556998],
       [103118.12522437, 103117.73008609, 103117.37004281, ...,
        103119.15402548, 103118.84183271, 103118.43235029],
       [103124.42631702, 103123.63402538, 103122.94472891, ...,
        103126.50412049, 103125.87494492, 103125.21183289],
       ...,
       [102171.71136794, 102174.91820481, 102178.10779961, ...,
        102162.19574897, 102165.38671717, 102168.58070383],
       [102109.38172899, 102110.10168872, 102110.78704101, ...,
        102107.28331347, 102107.99564363, 102108.6997948 ],
       [102001.79948493, 102001.98844932, 102002.13639802, ...,
        102001.42448674, 102001.5753651 , 102001.68522776]])}, 'TT': {'iFileName': 'TT', 'isSoilVariable': False, 'is3D': True, 'values': array([[[265.8